In [1]:
import numpy as np
import sys
import pandas as pd
from scipy import stats

Load the data: fragment files (bin 1kb); chromosomes coordinates; matrice in dense format (bin 1kb). The data are loaded with pandas, and the matrice is converted to an array with numpy so as to access specific data.

In [2]:
dir_seqs= "/mnt/d/Science/OwnCloud/Sequences/Genomes/"
out= "/mnt/d/Science/OwnCloud/Experiments/Capture/Quantif/"
input_dir = "/mnt/e/Science/ssHiC/Contact_files/"

#specify the path to the tab-delimited file containing chromosome name, length, and centromere location

#chrs = pd.read_csv(str(dir_seqs + "Yeast_genome_W303_Marcand/W303_710_chromosome_centromere_coordinates.tsv"), sep = "\t")
#chrs = pd.read_csv(str(dir_seqs + "Yeast_Genome_SK1_2019/sk1_chromosomes_centro_chr4_v1.tsv"), sep = "\t")
chrs = pd.read_csv(str(dir_seqs + "Yeast_genome_S288c_R64-2-1_2014/S288c_chr_centro_coordinates.tsv"), sep = "\t")
#print(chrs)


Load the data. It will show an array of integer (if the matrice is not pre-normalized). Sometimes tsv or csv. check

In [3]:
print(chrs)

      Chr   Length  Left_arm_length  Right_arm_length
0    chr1   230218           151465             78753
1    chr2   813184           238207            574977
2    chr3   316620           114385            202235
3    chr4  1531933           449711           1082222
4    chr5   576874           151987            424887
5    chr6   270161           148510            121651
6    chr7  1090940           497038            593902
7    chr8   562643           105703            456940
8    chr9   439888           355629             84259
9   chr10   745751           436425            309326
10  chr11   666816           440246            226570
11  chr12  1078177           150947            927230
12  chr13   924431           268031            656400
13  chr14   784333           628758            155575
14  chr15  1091291           326702            764589
15  chr16   948066           555957            392109


Normalize the data in the matrice by the total number of reads. It will return an array of floats.

In [4]:
samples = ["AD301_S288c_DSB_LY_Capture_artificial_cutsite_q30",
          "AD302_S288c_DSB_LY_Capture_artificial_cutsite_q30"]

for sample in samples:
    frags = pd.read_csv(str(input_dir + "Rebin/S288c/" + sample + "_1kb.frag.tsv"), sep = "\t")
    mat = pd.read_csv(str(input_dir + "Dense/S288c/" + sample + "_dense_1kb.mat.csv"), sep = " ", header = None)
    mat = np.array(mat)
    print(mat)
    #frags = frags.replace("chr4_v1", "chr4") 

    #print(sum(mat[:,:]))
    print("total number of contacts in the matrix =", sum(sum(mat)))
    mat_norm=mat/sum(sum(mat)) #matrix normalized


    #Define the lines in frags corresponding to each chromosomes in chrs.
    #Define the lines in frags of start and end positions of fragments corresponding to centromeres [-X kb to +X kb] around the cen-containing bin (= 5 bins total)
    #Define the lines in frags of the start and end positions of each telomere [consider a whole 20kb region]
    #chromosomes
    coord_chr_start = []
    coord_chr_end = []
    i = 0
    for i in chrs.loc[: ,"Chr"]:
        coord_chr_start.append(min(frags.loc[frags["chrom"]== str(i)].index.values))
        coord_chr_end.append(max(frags.loc[frags["chrom"]== str(i)].index.values))
    #print(coord_chr_start)
    #print(coord_chr_end)

    #centromeres
    #extra_bin = input("extra kb on each side of the centromeric bin?") #Normally 2 to have a 5kb window
    extra_bin = 14
    coord_cen_start = []
    coord_cen_end = []
    i = 0
    for i in chrs.loc[: ,"Chr"]:
        cen = int(chrs.loc[chrs["Chr"] == str(i)]["Left_arm_length"])
        cen_row=int(frags.loc[(frags["chrom"]== str(i)) & (frags["start_pos"] < cen) & (frags["end_pos"] > cen)].index.values)
        coord_cen_start.append(cen_row - int(extra_bin))
        coord_cen_end.append(cen_row + int(extra_bin) + 1)
    #print(coord_cen_start)
    #print(coord_cen_end)


    print("--------------------------")
    #Quantif telomeres contacts
    coord_tel_start = []
    coord_tel_end = []
    i = 0
    for i in chrs.loc[: ,"Chr"]:
        tel1 = 1+2
        tel2 = int(chrs.loc[chrs["Chr"] == str(i)]["Length"]) -1000
        tel1_row=int(frags.loc[(frags["chrom"]== str(i)) & (frags["start_pos"] < tel1) & (frags["end_pos"] > tel1)].index.values)
        coord_tel_start.append(tel1_row)
        coord_tel_end.append(tel1_row + 19)
        tel2_row=int(frags.loc[(frags["chrom"]== str(i)) & (frags["start_pos"] < tel2) & (frags["end_pos"] > tel2)].index.values)
        coord_tel_start.append(tel2_row - 19)
        coord_tel_end.append(tel2_row)
    #print(coord_tel_start)
    #print(coord_tel_end)
    print("done")


    #QUANTIF INTRA CONTACTS FOR EACH CHROMOSOMES
    print("--------------------------")
    output_dir = out + "Intra/"
    intra_contacts = []
    i=0
    while i < len(coord_chr_start):
        start = coord_chr_start[i]
        end = coord_chr_end[i]
        intra=0
        intra=sum(sum(mat_norm[start : end , start : end]))
        all_contacts=sum(sum(mat_norm[start : end , ]))
        intra_contacts.append(intra/all_contacts)
        i += 1
    print(intra_contacts)
    print("done")
    np.savetxt(output_dir + str(sample) + "_intra.tsv", intra_contacts, delimiter = "\t")



    print("--------------------------")
    #Quantify the contacts between the centromeres
    output_dir = out + "Centro/"
    cen_contacts = []
    i=0
    while i < len(coord_cen_start):
        j=0
        while j < len(coord_cen_start):
            if i != j:
                start1 = coord_cen_start[i]
                end1 = coord_cen_end[i]
                start2 = coord_cen_start[j]
                end2 = coord_cen_end[j]
                cen_contacts.append(sum(sum(mat_norm[start1 : end1 , start2 : end2])))
            j += 1
        i += 1
    #print(cen_contacts)
    print("done")
    np.savetxt(output_dir + str(sample) + "_centro_binsize_" + str(int(extra_bin) * 2 + 1) + "kb.tsv", cen_contacts, delimiter = "\t")



    print("--------------------------")
    #Quantify the contacts between CEN5 and other 15 centromeres
    output_dir = out + "Centro/"
    cen_contacts_chr5 = []
    i = 5
    j = 0
    while j < len(coord_cen_start):
        if i != j:
            start1 = coord_cen_start[i]
            end1 = coord_cen_end[i]
            start2 = coord_cen_start[j]
            end2 = coord_cen_end[j]
            cen_contacts_chr5.append(sum(sum(mat_norm[start1 : end1 , start2 : end2])))
        j += 1

    print(cen_contacts_chr5)
    print("done")
    np.savetxt(output_dir + str(sample) + "_chr5_centro_binsize_" + str(int(extra_bin) * 2 + 1) + "kb.tsv", cen_contacts_chr5, delimiter = "\t")



    print("--------------------------")
    #Quantify contcts between telomeres
    output_dir = out + "Telo/"
    tel_contacts = []
    i=0
    while i < len(coord_tel_start):
        j=0
        while j < len(coord_tel_end):
            if i != j:
                a = 0
                start1 = coord_tel_start[i]
                end1 = coord_tel_end[i]
                start2 = coord_tel_start[j]
                end2 = coord_tel_end[j]
                a = sum(sum(mat_norm[start1 : end1 , start2 : end2]))
                if a != 0:
                    tel_contacts.append(a)
            j += 1
        i += 1
    #print(tel_contacts)
    print("done")
    np.savetxt(output_dir + str(sample) + "_telo_20kb.tsv", tel_contacts, delimiter = "\t")


    print("--------------------------")
    #Quantify contacts between telomeres of chr5 and other telomeres
    output_dir = out + "Telo/"
    tel_contacts_chr5 = []
    for i in [11,12]:
        j=0
        while j < len(coord_tel_end):
            if i != j:
                start1 = coord_tel_start[i]
                end1 = coord_tel_end[i]
                start2 = coord_tel_start[j]
                end2 = coord_tel_end[j]
                tel_contacts_chr5.append(sum(sum(mat_norm[start1 : end1 , start2 : end2])))
            j += 1
    #print(tel_contacts_chr5)
    print("done")
    np.savetxt(output_dir + str(sample) + "_chr5_telo_10kb.tsv", tel_contacts_chr5, delimiter = "\t")


[[16 55 31 ...  0  0  0]
 [55  5 57 ...  0  0  0]
 [31 57 15 ...  0  0  0]
 ...
 [ 0  0  0 ...  6  4  7]
 [ 0  0  0 ...  4  0  0]
 [ 0  0  0 ...  7  0  0]]
total number of contacts in the matrix = 9300702
--------------------------
done
--------------------------
[0.7058082531802428, 0.8031741773834581, 0.7441081671201086, 0.8293291044141401, 0.7581818392751373, 0.7117608791732809, 0.7936748287783904, 0.7705047598194028, 0.7412714405875377, 0.7806330460841713, 0.7757567401115862, 0.8222550741525324, 0.7919848609276061, 0.7833170402599534, 0.8009147913991231, 0.7832200524000144]
done
--------------------------
done
--------------------------
[2.1288715625981783e-05, 2.4514278599615386e-05, 2.236390328385965e-05, 2.075112179704285e-05, 2.010600920231613e-05, 2.5374428725917677e-05, 2.2471422049647433e-05, 2.5912022554856612e-05, 3.773908679151316e-05, 2.741728527588563e-05, 2.365412847331309e-05, 2.107367809440621e-05, 2.096615932861842e-05, 2.1181196860193995e-05, 2.4406759833827598e-05

In [5]:
print(i)

chr4_v1


For DpnII only matrices

In [16]:
#samples = ["AD106", "AD107", "AD108", "AD109", "AD110", "AD111", "AD112", "AD113"]
#samples = ["AP51", "AP52"]
samples = ["AP36_AP37_merged", "AP38_AP39_merged"]

for sample in samples:
    frags = pd.read_csv(str(input_dir + "Rebin/" + sample + "_DpnII_1kb.frag.tsv"), sep = "\t")
    mat = pd.read_csv(str(input_dir + "Dense/" + sample + "_dense_DpnII_1kb.mat.csv"), sep = " ", header = None)
    mat = np.array(mat)
    print(mat)
    #frags = frags.replace("chr4_v1", "chr4") 

    #print(sum(mat[:,:]))
    print("total number of contacts in the matrix =", sum(sum(mat)))
    mat_norm=mat/sum(sum(mat)) #matrix normalized


    #Define the lines in frags corresponding to each chromosomes in chrs.
    #Define the lines in frags of start and end positions of fragments corresponding to centromeres [-X kb to +X kb] around the cen-containing bin (= 5 bins total)
    #Define the lines in frags of the start and end positions of each telomere [consider a whole 20kb region]
    #chromosomes
    coord_chr_start = []
    coord_chr_end = []
    i = 0
    for i in chrs.loc[: ,"Chr"]:
        coord_chr_start.append(min(frags.loc[frags["chrom"]== str(i)].index.values))
        coord_chr_end.append(max(frags.loc[frags["chrom"]== str(i)].index.values))
    #print(coord_chr_start)
    #print(coord_chr_end)

    #centromeres
    #extra_bin = input("extra kb on each side of the centromeric bin?") #Normally 2 to have a 5kb window
    extra_bin = 14
    coord_cen_start = []
    coord_cen_end = []
    i = 0
    for i in chrs.loc[: ,"Chr"]:
        cen = int(chrs.loc[chrs["Chr"] == str(i)]["Left_arm_length"])
        cen_row=int(frags.loc[(frags["chrom"]== str(i)) & (frags["start_pos"] < cen) & (frags["end_pos"] > cen)].index.values)
        coord_cen_start.append(cen_row - int(extra_bin))
        coord_cen_end.append(cen_row + int(extra_bin) + 1)
    #print(coord_cen_start)
    #print(coord_cen_end)


    print("--------------------------")
    #Quantif telomeres contacts
    coord_tel_start = []
    coord_tel_end = []
    i = 0
    for i in chrs.loc[: ,"Chr"]:
        tel1 = 1+2
        tel2 = int(chrs.loc[chrs["Chr"] == str(i)]["Length"]) -1000
        tel1_row=int(frags.loc[(frags["chrom"]== str(i)) & (frags["start_pos"] < tel1) & (frags["end_pos"] > tel1)].index.values)
        coord_tel_start.append(tel1_row)
        coord_tel_end.append(tel1_row + 19)
        tel2_row=int(frags.loc[(frags["chrom"]== str(i)) & (frags["start_pos"] < tel2) & (frags["end_pos"] > tel2)].index.values)
        coord_tel_start.append(tel2_row - 19)
        coord_tel_end.append(tel2_row)
    #print(coord_tel_start)
    #print(coord_tel_end)
    print("done")


    #QUANTIF INTRA CONTACTS FOR EACH CHROMOSOMES
    print("--------------------------")
    output_dir = out + "Intra/"
    intra_contacts = []
    i=0
    while i < len(coord_chr_start):
        start = coord_chr_start[i]
        end = coord_chr_end[i]
        intra=0
        intra=sum(sum(mat_norm[start : end , start : end]))
        all_contacts=sum(sum(mat_norm[start : end , ]))
        intra_contacts.append(intra/all_contacts)
        i += 1
    print(intra_contacts)
    print("done")
    np.savetxt(output_dir + str(sample) + "_intra.tsv", intra_contacts, delimiter = "\t")



    print("--------------------------")
    #Quantify the contacts between the centromeres
    output_dir = out + "Centro/"
    cen_contacts = []
    i=0
    while i < len(coord_cen_start):
        j=0
        while j < len(coord_cen_start):
            if i != j:
                start1 = coord_cen_start[i]
                end1 = coord_cen_end[i]
                start2 = coord_cen_start[j]
                end2 = coord_cen_end[j]
                cen_contacts.append(sum(sum(mat_norm[start1 : end1 , start2 : end2])))
            j += 1
        i += 1
    #print(cen_contacts)
    print("done")
    np.savetxt(output_dir + str(sample) + "_centro_binsize_" + str(int(extra_bin) * 2 + 1) + "kb.tsv", cen_contacts, delimiter = "\t")



    print("--------------------------")
    #Quantify the contacts between CEN5 and other 15 centromeres
    output_dir = out + "Centro/"
    cen_contacts_chr5 = []
    i = 5
    j = 0
    while j < len(coord_cen_start):
        if i != j:
            start1 = coord_cen_start[i]
            end1 = coord_cen_end[i]
            start2 = coord_cen_start[j]
            end2 = coord_cen_end[j]
            cen_contacts_chr5.append(sum(sum(mat_norm[start1 : end1 , start2 : end2])))
        j += 1

    print(cen_contacts_chr5)
    print("done")
    np.savetxt(output_dir + str(sample) + "_chr5_centro_binsize_" + str(int(extra_bin) * 2 + 1) + "kb.tsv", cen_contacts_chr5, delimiter = "\t")



    print("--------------------------")
    #Quantify contcts between telomeres
    output_dir = out + "Telo/"
    tel_contacts = []
    i=0
    while i < len(coord_tel_start):
        j=0
        while j < len(coord_tel_end):
            if i != j:
                a = 0
                start1 = coord_tel_start[i]
                end1 = coord_tel_end[i]
                start2 = coord_tel_start[j]
                end2 = coord_tel_end[j]
                a = sum(sum(mat_norm[start1 : end1 , start2 : end2]))
                if a != 0:
                    tel_contacts.append(a)
            j += 1
        i += 1
    #print(tel_contacts)
    print("done")
    np.savetxt(output_dir + str(sample) + "_telo_20kb.tsv", tel_contacts, delimiter = "\t")


    print("--------------------------")
    #Quantify contacts between telomeres of chr5 and other telomeres
    output_dir = out + "Telo/"
    tel_contacts_chr5 = []
    for i in [11,12]:
        j=0
        while j < len(coord_tel_end):
            if i != j:
                start1 = coord_tel_start[i]
                end1 = coord_tel_end[i]
                start2 = coord_tel_start[j]
                end2 = coord_tel_end[j]
                tel_contacts_chr5.append(sum(sum(mat_norm[start1 : end1 , start2 : end2])))
            j += 1
    #print(tel_contacts_chr5)
    print("done")
    np.savetxt(output_dir + str(sample) + "_chr5_telo_10kb.tsv", tel_contacts_chr5, delimiter = "\t")


[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
total number of contacts in the matrix = 44095939
--------------------------
done
--------------------------
[0.5718245878571194, 0.6095497736679477, 0.5841790820689022, 0.6707477657610017, 0.5805378969287079, 0.5804405050205328, 0.63006794811416, 0.5948712942568966, 0.585256185075522, 0.6198645545646377, 0.6052738830233043, 0.6756934306770003, 0.6268063410157241, 0.6071886689501065, 0.6353966211783003, 0.6161534180572109]
done
--------------------------
done
--------------------------
[3.610309783855606e-05, 3.440226094289544e-05, 3.007079631527974e-05, 3.270142404723483e-05, 3.524134047808801e-05, 3.1182009753778006e-05, 3.583096393525037e-05, 3.7894646035318584e-05, 3.596703088690323e-05, 3.8892470347439476e-05, 3.1499499307634665e-05, 3.24519679692046e-05, 3.4084771389038794e-05, 3.594435306162773e-05, 3.0501674995513756e-05]
done
--------------------------
done
-

In [10]:
print(tel1)
print(tel2)
print(str(i))
print(frags.loc[(frags["chrom"]== str(i)) & (frags["start_pos"] < tel2)])

3
828469
chr2
       id chrom  start_pos  end_pos   size  gc_content
229     1  chr2          0     1000   1000    0.460091
230     2  chr2       1000     2000   1000    0.438095
231     3  chr2       2000     3000   1000    0.451932
232     4  chr2       3000     4000   1000    0.452892
233     5  chr2       4000     5000   1000    0.389157
...   ...   ...        ...      ...    ...         ...
1053  825  chr2     824000   804833 -19167    0.472953
1054  826  chr2     825000   804833 -20167    0.506568
1055  827  chr2     826000   805833 -20167         NaN
1056  828  chr2     827000   806833 -20167         NaN
1057  829  chr2     828000   804833 -23167    0.467177

[829 rows x 6 columns]


Define the lines in frags corresponding to each chromosomes in chrs.

Define the lines in frags of start and end positions of fragments corresponding to centromeres [-X kb to +X kb] around the cen-containing bin (= 5 bins total)

Define the lines in frags of the start and end positions of each telomere [consider a whole 20kb region]

In [111]:
#chromosomes
coord_chr_start = []
coord_chr_end = []
i = 0
for i in range(0, len(chrs)):
    coord_chr_start.append(min(frags.loc[frags["chrom"]== "chr" + str(i + 1)].index.values))
    coord_chr_end.append(max(frags.loc[frags["chrom"]== "chr" + str(i + 1)].index.values))
#print(coord_chr_start)
#print(coord_chr_end)

#centromeres
#extra_bin = input("extra kb on each side of the centromeric bin?") #Normally 2 to have a 5kb window
extra_bin = 2
coord_cen_start = []
coord_cen_end = []
i = 0
for i in range(0, len(chrs)):
    cen = int(chrs.loc[chrs["Chr"] == "chr" + str(i+1)]["Left_arm_length"])
    cen_row=int(frags.loc[(frags["chrom"]== "chr" + str(i + 1)) & (frags["start_pos"] < cen) & (frags["end_pos"] > cen)].index.values)
    coord_cen_start.append(cen_row - int(extra_bin))
    coord_cen_end.append(cen_row + int(extra_bin) + 1)
#print(coord_cen_start)
#print(coord_cen_end)

#telomeres
coord_tel_start = []
coord_tel_end = []
i = 0
for i in range(0, len(chrs)):
    tel1 = 1+2
    tel2 = int(chrs.loc[chrs["Chr"] == "chr" + str(i+1)]["Length"]) -1000
    tel1_row=int(frags.loc[(frags["chrom"]== "chr" + str(i + 1)) & (frags["start_pos"] < tel1) & (frags["end_pos"] > tel1)].index.values)
    coord_tel_start.append(tel1_row)
    coord_tel_end.append(tel1_row + 19)
    tel2_row=int(frags.loc[(frags["chrom"]== "chr" + str(i + 1)) & (frags["start_pos"] < tel2) & (frags["end_pos"] > tel2)].index.values)
    coord_tel_start.append(tel2_row - 19)
    coord_tel_end.append(tel2_row)
#print(coord_tel_start)
#print(coord_tel_end)
print("done")

done


Quantify the intra-chromosomal contacts

In [112]:
output_dir = out + "Intra/"
intra_contacts = []
i=0
while i < len(coord_chr_start):
    start = coord_chr_start[i]
    end = coord_chr_end[i]
    intra=0
    intra=sum(sum(mat_norm[start : end , start : end]))
    all_contacts=sum(sum(mat_norm[start : end , ]))
    intra_contacts.append(intra/all_contacts)
    i += 1
print(intra_contacts)
print("done")
np.savetxt(output_dir + str(sample) + "_intra.tsv", intra_contacts, delimiter = "\t")


[0.7114501551015645, 0.8076396302773152, 0.7419116190507897, 0.8468769415228061, 0.7728027290857843, 0.7270359803152728, 0.81747645369735, 0.7746843288935392, 0.756927915447601, 0.7938471992285933, 0.7906909439703667, 0.8323624270277716, 0.8113514389111354, 0.8008498666969448, 0.816766416005933, 0.8057428076145948]
done


Quantify the contacts between the centromeres

In [113]:
output_dir = out + "Centro/"
cen_contacts = []
i=0
while i < len(coord_cen_start):
    j=0
    while j < len(coord_cen_start):
        if i != j:
            start1 = coord_cen_start[i]
            end1 = coord_cen_end[i]
            start2 = coord_cen_start[j]
            end2 = coord_cen_end[j]
            cen_contacts.append(sum(sum(mat_norm[start1 : end1 , start2 : end2])))
        j += 1
    i += 1
#print(cen_contacts)
print("done")
np.savetxt(output_dir + str(sample) + "_centro_binsize_" + str(int(extra_bin) * 2 + 1) + "kb.tsv", cen_contacts, delimiter = "\t")


done


Quantify the contact of Cen 5

In [114]:
output_dir = out + "Centro/"
cen_contacts_chr5 = []
i = 5
j = 0
while j < len(coord_cen_start):
    if i != j:
        start1 = coord_cen_start[i]
        end1 = coord_cen_end[i]
        start2 = coord_cen_start[j]
        end2 = coord_cen_end[j]
        cen_contacts_chr5.append(sum(sum(mat_norm[start1 : end1 , start2 : end2])))
    j += 1

print(cen_contacts_chr5)
print("done")
np.savetxt(output_dir + str(sample) + "_chr5_centro_binsize_" + str(int(extra_bin) * 2 + 1) + "kb.tsv", cen_contacts_chr5, delimiter = "\t")


[1.98174805151351e-06, 2.7891268873153096e-06, 2.1285442034774737e-06, 2.079612152822819e-06, 2.006214076840837e-06, 2.177476254132128e-06, 2.4466025327327283e-06, 2.2753403554414376e-06, 1.8838839502042009e-06, 2.1040781781501465e-06, 1.467961519639637e-06, 1.467961519639637e-06, 2.0796121528228192e-06, 2.1040781781501465e-06, 2.0796121528228192e-06]
done


Quantify the contacts between telomeres

In [115]:
output_dir = out + "Telo/"
tel_contacts = []
i=0
while i < len(coord_tel_start):
    j=0
    while j < len(coord_tel_end):
        if i != j:
            a = 0
            start1 = coord_tel_start[i]
            end1 = coord_tel_end[i]
            start2 = coord_tel_start[j]
            end2 = coord_tel_end[j]
            a = sum(sum(mat_norm[start1 : end1 , start2 : end2]))
            if a != 0:
                tel_contacts.append(a)
        j += 1
    i += 1
#print(tel_contacts)
print("done")
np.savetxt(output_dir + str(sample) + "_telo_20kb.tsv", tel_contacts, delimiter = "\t")


done


Quantify the contacts made between chr5 telomeres with others 

In [116]:
output_dir = out + "Telo/"
tel_contacts_chr5 = []
for i in [11,12]:
    j=0
    while j < len(coord_tel_end):
        if i != j:
            start1 = coord_tel_start[i]
            end1 = coord_tel_end[i]
            start2 = coord_tel_start[j]
            end2 = coord_tel_end[j]
            tel_contacts_chr5.append(sum(sum(mat_norm[start1 : end1 , start2 : end2])))
        j += 1
#print(tel_contacts_chr5)
print("done")
np.savetxt(output_dir + str(sample) + "_chr5_telo_10kb.tsv", tel_contacts_chr5, delimiter = "\t")


done
